Topics Covered:
    1. Using Group by
    2. Simple Window function uses
    3. Using Order By
    4. Using rowsbetween
    5. taking only two records in list
    6. Using all aggregates with Window function

In [1]:
!pip install -q findspark

import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import Window

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [6]:
l = [
    (1,"sales",4200),
    (2,"admin",3100),
    (3,"sales",4000),
    (4,"sales",4000),
    (5,"admin",2700),
    (6,"dev",3400),
    (7,"dev",5200),
    (8,"dev",3700),
    (9,"dev",4400),
    (10,"dev",4400)
]
data = spark.createDataFrame(l,schema=["id","dept","salary"])
data.show()

+---+-----+------+
| id| dept|salary|
+---+-----+------+
|  1|sales|  4200|
|  2|admin|  3100|
|  3|sales|  4000|
|  4|sales|  4000|
|  5|admin|  2700|
|  6|  dev|  3400|
|  7|  dev|  5200|
|  8|  dev|  3700|
|  9|  dev|  4400|
| 10|  dev|  4400|
+---+-----+------+



In [7]:
#01. Using Group by:
df = data.groupBy("dept").agg(
    F.expr("collect_list(salary)").alias("list_salary"),
    F.expr("avg(salary)").alias("avg"),
    F.expr("sum(salary)").alias("sum"))
df.show()

+-----+--------------------+------------------+-----+
| dept|         list_salary|               avg|  sum|
+-----+--------------------+------------------+-----+
|  dev|[3400, 5200, 3700...|            4220.0|21100|
|sales|  [4200, 4000, 4000]|4066.6666666666665|12200|
|admin|        [3100, 2700]|            2900.0| 5800|
+-----+--------------------+------------------+-----+



In [14]:
#02. Simple Window function uses.
windowSpec = Window.partitionBy("dept")
df = data.withColumn("list_salary",F.collect_list(F.col("salary")).over(windowSpec))\
.withColumn("avg",F.avg(F.col("salary")).over(windowSpec))

df.show()

+---+-----+------+--------------------+------------------+
| id| dept|salary|         list_salary|               avg|
+---+-----+------+--------------------+------------------+
|  6|  dev|  3400|[3400, 5200, 3700...|            4220.0|
|  7|  dev|  5200|[3400, 5200, 3700...|            4220.0|
|  8|  dev|  3700|[3400, 5200, 3700...|            4220.0|
|  9|  dev|  4400|[3400, 5200, 3700...|            4220.0|
| 10|  dev|  4400|[3400, 5200, 3700...|            4220.0|
|  1|sales|  4200|  [4200, 4000, 4000]|4066.6666666666665|
|  3|sales|  4000|  [4200, 4000, 4000]|4066.6666666666665|
|  4|sales|  4000|  [4200, 4000, 4000]|4066.6666666666665|
|  2|admin|  3100|        [3100, 2700]|            2900.0|
|  5|admin|  2700|        [3100, 2700]|            2900.0|
+---+-----+------+--------------------+------------------+



In [16]:
#03. Using Order By:
windowSpec = Window.partitionBy("dept").orderBy(F.asc("salary"))
df = data.withColumn("list_salary",F.collect_list(F.col("salary")).over(windowSpec))\
.withColumn("avg",F.avg(F.col("salary")).over(windowSpec))

df.show()

+---+-----+------+--------------------+------------------+
| id| dept|salary|         list_salary|               avg|
+---+-----+------+--------------------+------------------+
|  6|  dev|  3400|              [3400]|            3400.0|
|  8|  dev|  3700|        [3400, 3700]|            3550.0|
|  9|  dev|  4400|[3400, 3700, 4400...|            3975.0|
| 10|  dev|  4400|[3400, 3700, 4400...|            3975.0|
|  7|  dev|  5200|[3400, 3700, 4400...|            4220.0|
|  3|sales|  4000|        [4000, 4000]|            4000.0|
|  4|sales|  4000|        [4000, 4000]|            4000.0|
|  1|sales|  4200|  [4000, 4000, 4200]|4066.6666666666665|
|  5|admin|  2700|              [2700]|            2700.0|
|  2|admin|  3100|        [2700, 3100]|            2900.0|
+---+-----+------+--------------------+------------------+



In [17]:
#04. using rowsBetween:
windowSpec = Window.partitionBy("dept").orderBy(F.asc("salary")).rowsBetween(Window.unboundedPreceding, Window.currentRow)
df = data.withColumn("list_salary",F.collect_list(F.col("salary")).over(windowSpec))\
.withColumn("avg",F.avg(F.col("salary")).over(windowSpec))

df.show()

+---+-----+------+--------------------+------------------+
| id| dept|salary|         list_salary|               avg|
+---+-----+------+--------------------+------------------+
|  6|  dev|  3400|              [3400]|            3400.0|
|  8|  dev|  3700|        [3400, 3700]|            3550.0|
|  9|  dev|  4400|  [3400, 3700, 4400]|3833.3333333333335|
| 10|  dev|  4400|[3400, 3700, 4400...|            3975.0|
|  7|  dev|  5200|[3400, 3700, 4400...|            4220.0|
|  3|sales|  4000|              [4000]|            4000.0|
|  4|sales|  4000|        [4000, 4000]|            4000.0|
|  1|sales|  4200|  [4000, 4000, 4200]|4066.6666666666665|
|  5|admin|  2700|              [2700]|            2700.0|
|  2|admin|  3100|        [2700, 3100]|            2900.0|
+---+-----+------+--------------------+------------------+



In [18]:
#05. taking only two records in list:
windowSpec = Window.partitionBy("dept").orderBy(F.asc("salary")).rowsBetween(-1,Window.currentRow)

df = data.withColumn("list_salary",F.collect_list(F.col("salary")).over(windowSpec))\
.withColumn("avg",F.avg(F.col("salary")).over(windowSpec))

df.show()

+---+-----+------+------------+------+
| id| dept|salary| list_salary|   avg|
+---+-----+------+------------+------+
|  6|  dev|  3400|      [3400]|3400.0|
|  8|  dev|  3700|[3400, 3700]|3550.0|
|  9|  dev|  4400|[3700, 4400]|4050.0|
| 10|  dev|  4400|[4400, 4400]|4400.0|
|  7|  dev|  5200|[4400, 5200]|4800.0|
|  3|sales|  4000|      [4000]|4000.0|
|  4|sales|  4000|[4000, 4000]|4000.0|
|  1|sales|  4200|[4000, 4200]|4100.0|
|  5|admin|  2700|      [2700]|2700.0|
|  2|admin|  3100|[2700, 3100]|2900.0|
+---+-----+------+------------+------+



In [19]:
#06. Using all aggregates.
windowSpec = Window.partitionBy("dept").orderBy(F.asc("salary"))

df = (
    data.withColumn("avg_salary",F.avg(F.col("salary")).over(windowSpec))
    .withColumn("total",F.sum(F.col("salary")).over(windowSpec))
    .withColumn("rank",F.rank().over(windowSpec))
    .withColumn("dense_rank",F.dense_rank().over(windowSpec))
    .withColumn("percent_rank",F.percent_rank().over(windowSpec))
)
df.show()

+---+-----+------+------------------+-----+----+----------+------------+
| id| dept|salary|        avg_salary|total|rank|dense_rank|percent_rank|
+---+-----+------+------------------+-----+----+----------+------------+
|  6|  dev|  3400|            3400.0| 3400|   1|         1|         0.0|
|  8|  dev|  3700|            3550.0| 7100|   2|         2|        0.25|
|  9|  dev|  4400|            3975.0|15900|   3|         3|         0.5|
| 10|  dev|  4400|            3975.0|15900|   3|         3|         0.5|
|  7|  dev|  5200|            4220.0|21100|   5|         4|         1.0|
|  3|sales|  4000|            4000.0| 8000|   1|         1|         0.0|
|  4|sales|  4000|            4000.0| 8000|   1|         1|         0.0|
|  1|sales|  4200|4066.6666666666665|12200|   3|         2|         1.0|
|  5|admin|  2700|            2700.0| 2700|   1|         1|         0.0|
|  2|admin|  3100|            2900.0| 5800|   2|         2|         1.0|
+---+-----+------+------------------+-----+----+---